Erstelle eine neue Conda Umgebung wanderwege

In [7]:
conda create --name wanderwege
conda activate wanderwege

Channels:
 - defaults
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - wanderwege
  - activate

Current channels:

  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


Wir benutzen Python 3.12.2 mit folenden Paketen

In [ ]:
conda install pandas
conda install -c anaconda sqlalchemy
conda install -c conda-forge overpy
conda install -c anaconda mysql-connector-python
conda install -c conda-forge pymysql
pip install overpy

In [23]:
import overpy
import pandas as pd

# Initialisiere die Overpass API mit einer benutzerdefinierten URL
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

# Overpass Query für Wanderwege innerhalb der Schweiz. Mit Center erhalten wir die Koordinaten in der Mitte eines Wnaderweges
query = """
[out:json][timeout:25];
relation["route"="hiking"]["name"]["name"!~"fixme", i]["network"="lwn"](46.8167, 5.9559, 47.8085, 10.4921);
out center tags;
"""

# Führe die Anfrage aus
result = api.query(query)

# Liste zum Speichern der extrahierten Informationen für das DataFrame
data = []

# Iteration über alle relations
for relation in result.relations:
    
    # Center ist ein Tupel mit langitute und longitude, wir möchten nur einen wert
    lat = getattr(relation, 'center_lat')
    lon = getattr(relation, 'center_lon')

    # Füge die Relation-Daten zu einer Liste hinzu,
    relation_data = {
    'id': relation.id,
    'name': relation.tags.get('name'),
    'lat': lat,
    'lon': lon
    }

    # Jedes Dictionary wird in die Liste 'data' eingefügt, sodass die Daten gesammelt werden
    data.append(relation_data)

# Wenn alle Relationen verarbeitet wurden, erstelle das DataFrame
df = pd.DataFrame(data)

# Zeige das DataFrame an
print(df)


         id                             name          lat          lon
0  17666736  Nasse Küche - Siegenbachmündung  -21.6557036  -96.1460220


Die Resultate sind nun in einem Dataframe und können in die mysql Datenbank gespeichert werden, 
dazu erstelle ich die db

In [20]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd=""
)

print(mydb)

Nun lade ich die Daten


In [21]:
import pandas as pd
from sqlalchemy import create_engine

# Verbindung zur MySQL-Datenbank herstellen (ersetze Platzhalter mit deinen Daten)
user = 'root'        # MySQL-Benutzername
password = 'mysqlfortuna'  # MySQL-Passwort
host = '127.0.0.1'   # Server (bei lokalen Installationen ist das oft 'localhost')
database = 'mydatabase'  # Name der Datenbank, in die du schreiben möchtest

# Erstelle die Verbindungs-Engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# DataFrame in die MySQL-Datenbank-Tabelle schreiben (wenn die Tabelle nicht existiert, wird sie erstellt)
df.to_sql(name='Wanderwege', con=engine, if_exists='replace', index=False)

print("DataFrame erfolgreich in die MySQL-Datenbank geladen!")

DataFrame erfolgreich in die MySQL-Datenbank geladen!
